# Kryptools

## Zmod: Ring of integers modulo n

To define a finite Galois field modulo the prime $5$ use

In [1]:
from kryptools import Zmod

gf = Zmod(5)

To declare $3$ as an element of our Galois field use

In [2]:
gf(3)

3

Basic arithmetic operations work as expected:

In [3]:
gf(3) + gf(2)

0

In [4]:
gf(3) ** -1

2

To determine the multiplicative [order](https://en.wikipedia.org/wiki/Order_(group_theory)) of an element in the group $\mathbb{Z}_n^*$ use:

In [5]:
gf(3).order()

4

You can also check if an element is a [generator](https://en.wikipedia.org/wiki/Primitive_root_modulo_n) (a.k.a primitive root) fo $\mathbb{Z}_n^*$:

In [6]:
gf(3).is_generator()

True

If you set the option `short` to `False` then $\pmod{5}$ will be added:

In [7]:
gf.short = False
gf(6)

1 (mod 5)

## Primes

A tuple of primes below a bound $B$ can be obtained using the [sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes):

In [8]:
from kryptools import sieve_eratosthenes

sieve_eratosthenes(20)

(2, 3, 5, 7, 11, 13, 17, 19)

To test if a number is probable prime you can use

In [9]:
from kryptools import is_prime

is_prime(17)

True

For numbers below $3317044064679887385961981$ it performs a deterministic variant of the [Miller-Rabin test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test)

In [10]:
from kryptools.primes import miller_rabin_test

miller_rabin_test(17, [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41])

True

Otherwise it combines a Miller-Rabin test with base $2$ with a strong Lucas test ([Baillie–PSW primality test](https://en.wikipedia.org/wiki/Baillie%E2%80%93PSW_primality_test)).

The next prime larger or equal to a given number

In [11]:
from kryptools import next_prime

next_prime(100)

101

A random prime of given bit length

In [12]:
from kryptools import random_prime

p = random_prime(128)
assert is_prime(p)
p.bit_length()

128

Similarly, a random [safe prime](https://en.wikipedia.org/wiki/Safe_and_Sophie_Germain_primes) p of given bit length with and ord(2)=(p-1)/2

In [13]:
from kryptools import random_safeprime, is_safeprime

p = random_safeprime(128)
assert is_safeprime(p)
p.bit_length()

128

Finally a random [strong prime](https://en.wikipedia.org/wiki/Strong_prime) $p$ with factors of $p\pm 1$ of given bit length using Gordon's algorithm:

In [14]:
from kryptools import random_strongprime

t, s, r, p = random_strongprime(128)
assert is_prime(p) and is_prime(t) and is_prime(s) and is_prime(r)
assert (p + 1) % s == 0 and (p - 1) % r == 0 and (r - 1) % t == 0
p.bit_length()

271

## Number theory

To perform the [extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) use

In [15]:
from kryptools.nt import egcd

a, b = 7, 8
g, x, y = egcd(a, b)

assert x * a + y * b == g
g, x, y

(1, -1, 1)

To solve a list of congruences using the [Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) use

In [16]:
from kryptools.nt import crt

a, m = [1, 2, 3], [2, 3, 5]
sol = crt(a, m)

assert [sol % p for p in m] == a
sol

23

To compute the continued fraction expansion of a rational number use

In [17]:
from fractions import Fraction
from kryptools import cf, convergents

cf(Fraction(18, 23))

[0, 1, 3, 1, 1, 2]

To get the convergents use

In [18]:
convergents(_)

[0, 1, 3/4, 4/5, 7/9, 18/23]

To compute the [Jacobi symbol](https://en.wikipedia.org/wiki/Jacobi_symbol) use

In [19]:
from kryptools import jacobi_symbol

jacobi_symbol(4, 7)

1

Since it equals $+1$ the number is a [quadratic residue](https://en.wikipedia.org/wiki/Quadratic_residue) and has a square root modulo the prime $7$

In [20]:
from kryptools import sqrt_mod

sqrt_mod(4, 7)

2

To compute [Euler's $\varphi$ function](https://en.wikipedia.org/wiki/Euler%27s_totient_function) (a.k.a totient function) or [Carmichael's $\lambda$ function](https://en.wikipedia.org/wiki/Carmichael_function) use

In [21]:
from kryptools.nt import carmichael_lambda, euler_phi

n = 100
euler_phi(n), carmichael_lambda(n)

(40, 20)

To compute the order of an element of the [multiplicative group of integers modulo $n$](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n), $\mathbb{Z}_n^*$, use

In [22]:
from kryptools.nt import order

order(5, 7)

6

## Factoring integers

To factor an integer $n$ into its prime factors use

In [23]:
from kryptools import factorint

n = 2**128 + 1
factorint(n, verbose=1)

Trial division found: []
Factors found ( ecm ):  [59649589127497217, 5704689200685129054721]


{59649589127497217: 1, 5704689200685129054721: 1}

You can also use a specific algorithm:

Fermat's method

In [24]:
from kryptools.factor_fmt import factor_fermat

factor_fermat(90003)

[3, 19, 1579]

Pollard $p-1$

In [25]:
from kryptools.factor_pm1 import factor_pm1

factor_pm1(832283)

487

Lentstra's ECM  (if no factor is found, `None` is returned)

In [26]:
from kryptools.factor_ecm import factor_ecm

factor_ecm(832283)

A variant of Dixon's method

In [27]:
from kryptools.factor_dix import factor_dixon

factor_dixon(832283)

1709

Quadratic sieve (with a single polynomial)

In [28]:
from kryptools.factor_qs import factor_qs

factor_qs(832283)

1709

## Discrete logarithms

To solve a [discrete log](https://en.wikipedia.org/wiki/Discrete_logarithm) problem use

In [29]:
from random import randint

from kryptools import dlog

p, m, a = [557639, 278819, 2]  # m is the order of a in Z_p
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p) == x

You can also use a specific algorithm:

Exhaustive search:

In [30]:
from kryptools.dlp import dlog_naive

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p, m) == x

Pollard $\rho$ (with Brent's or Floyd's cycle detection algorithm)

In [31]:
from kryptools.dlp_rho import dlog_rho

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_rho(a, b, p, m, brent = True) == x

Shanks's baby step/giant step

In [32]:
from kryptools.dlp_bsgs import dlog_bsgs

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_bsgs(a, b, p, m) == x

Index calculus

In [33]:
from kryptools.dlp_ic import dlog_ic

p, m, a = [24570203447, 12285101723, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_ic(a, b, p, m, verbose = 1) == x

Factorbase: bound = 359, size = 72, max_trys = 3000
Success after 69 relations out of 73.


Quadratic sieve

In [34]:
from kryptools.dlp_qs import dlog_qs

p, m, a = [28031135240181527, 14015567620090763, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_qs(a, b, p, m, verbose = 1) == x

Factorbase: bound = 1903,  size = 291 + 470 = 761, max_trys = 32530
Done sieving.
Success after 476 relations out of 762.


## Linear Algebra

The basic class for working with matrices:

In [35]:
from kryptools.la import Matrix

M = Matrix([[1, 2, 3], [3, 4, 5]])
N = Matrix([[1], [3], [4]])
M * N

[ 19 ]
[ 35 ]

Slicing to access (e.g.) rows:

In [36]:
M[:, 0]

[ 1 ]
[ 3 ]

Also works with assignments, e.g., to swap two columns:

In [37]:
M[0, :], M[1, :] = M[1, :], M[0, :]
M

[ 3, 4, 5 ]
[ 1, 2, 3 ]

The coefficients can be from a field, e.g. `Fractions`. The `map` method applies a given function to all elements. Here we compute the [row reduced echelon form](https://en.wikipedia.org/wiki/Row_echelon_form).

In [38]:
from fractions import Fraction

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]])
M.map(Fraction)
print(M)
M.rref()

[ 1, 2,  3 ]
[ 4, 5,  6 ]
[ 7, 8, 12 ]


[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

The [determinant](https://en.wikipedia.org/wiki/Determinant)

In [39]:
print(M.det())

-9


or the [inverse](https://en.wikipedia.org/wiki/Invertible_matrix)

In [40]:
Mi = M.inv()
print(Mi)
M * Mi

[ -4/3,    0,  1/3 ]
[  2/3,    1, -2/3 ]
[  1/3, -2/3,  1/3 ]


[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

Same works if the coefficients are from a finite field or ring:

In [41]:
gf = Zmod(11, short = True)

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]], ring = gf)
M.det()

2

In [42]:
Mi = M.inv()
print(Mi)
M * Mi

[ 6, 0, 4 ]
[ 8, 1, 3 ]
[ 4, 3, 4 ]


[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

## Lattice

To compute the [Hermite normal form](https://en.wikipedia.org/wiki/Hermite_normal_form) use

In [43]:
from kryptools import hermite_nf

M = Matrix([[1, 2, 3, 1], [4, 5, 6, 4], [7, -8, 0, 7]])
hermite_nf(M)

[ 69, 58, 36 ]
[  0,  1,  0 ]
[  0,  0,  1 ]

To compute the [Gram-Schmidt decomposition](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) use

In [44]:
from kryptools import gram_schmidt

V = Matrix([[5, 8], [0, 1]], ring = Fraction)
Vs, M = gram_schmidt(V)
assert V == Vs * M
Vs

[ 5, 0 ]
[ 0, 1 ]

To compute the [lattice](https://en.wikipedia.org/wiki/Lattice_(group)) determinat/[gram determinant](https://en.wikipedia.org/wiki/Gram_matrix) use

In [45]:
from kryptools import gram_det

gram_det(V)

5.0

To compute the Hadamard ratio use

In [46]:
from kryptools import hadamard_ratio

hadamard_ratio(V)

0.3521856535823236

Solving the closest vector problem using Babai's rounding algorithm

In [47]:
from kryptools import babai_round_cvp
V = Matrix([[5, 8], [0, 1]])
x = Matrix([5.2, 0.4])

a = babai_round_cvp(x, V)
print((a - x).norm())
a

5.215361924162119


[ 0 ]
[ 0 ]

Using Lagrange-Gauß reduction gives a better solution

In [48]:
from kryptools import lagrange_lr

U=lagrange_lr(V)
a = babai_round_cvp(x, U)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

Alternatively we can use Babai's cosest plane algorithm

In [49]:
from kryptools import babai_plane_cvp
V = Matrix([[5, 8], [0, 1]])

a = babai_plane_cvp(x, V)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

LLL reduction

In [50]:
from kryptools import lll

V = Matrix([[1, 1, 2], [1, 0, -2], [3, 4, 5]])
U = lll(V)
U

[  0,  1, 2 ]
[ -1,  0, 2 ]
[  1, -1, 1 ]

To get a random unimodular matrix use

In [51]:
from kryptools import random_unimodular_matrix

random_unimodular_matrix(3, max_val = 9)

[ 1, -8, -9 ]
[ 0,  3,  4 ]
[ 4,  2,  9 ]

## Polynomials

A basic class for polynomials, entered as a list of coefficients

In [52]:
from kryptools import Poly

Poly([1, 2, 3])

3 x^2 + 2 x + 1

Computations in a polynomial ring can be done as follows:

Creating a public/private key pair suitable for [Kyber](https://en.wikipedia.org/wiki/Kyber)

In [53]:
from random import binomialvariate, randint, seed

seed(0)

p = 97
n = 4
k = 2
gf = Zmod(p)
gf.short = True


def kyber(n: int) -> list:
    return [1] + [0] * (n - 1) + [1]


def PolyKyber(c: list) -> "Poly":
    return Poly(c, ring = gf, modulus = kyber(n))

The matrix

In [54]:
A = Matrix(
    [
        [PolyKyber([randint(0, p - 1) for _ in range(n)]) for i in range(k)]
        for j in range(k)
    ]
)
A

[  33 x^3 + 5 x^2 + 53 x + 49, 38 x^3 + 51 x^2 + 62 x + 65 ]
[ 27 x^3 + 74 x^2 + 45 x + 61, 17 x^3 + 36 x^2 + 17 x + 64 ]

The small solution

In [55]:
s = Matrix([ PolyKyber([randint(-1,1) for _ in range(n)]) for i in range(k)])
s

[   x^3 + x + 96 ]
[ 96 x^2 + x + 1 ]

The error

In [56]:
q = p // (4 * k * n)
e = Matrix([PolyKyber([binomialvariate(2 * q) - q for _ in range(n)]) for i in range(k)])
e

[ 96 x^3 + x + 95 ]
[             2 x ]

The inhomogenous vector

In [57]:
b = A * s + e
b

[ 47 x^3 + 63 x^2 + 60 x + 38 ]
[ 47 x^3 + 30 x^2 + 42 x + 47 ]

## Eliptic curves

To declare an [elliptic curve](https://en.wikipedia.org/wiki/Elliptic_curve) in Weierstrass normal form use

In [58]:
from kryptools import EC_Weierstrass

ec = EC_Weierstrass(239, 3, 1)
ec.info()
print("Order:", ec.order())

Weierstrass curve y^2 = x^3 + 3 x + 1 over Z_239.
Order: 258


The order is computed using the Shanks-Maestre algorithm.

Basic arithmetic

In [59]:
P = ec.random()
Q = ec.random()
O = ec(None, None)  # point at infinity
P + Q

(113, 91)

To check if a point is on the curve use

In [60]:
P in ec

True

To compute the order of a point use

In [61]:
assert Q.order() * Q == O
Q.order()

129

The $n$'th division polynomial $\Psi_n(x,y)$ will anihilate all points whose order is a multiple of $n$:

In [62]:
P.psi(P.order())

0

To compute a discrete log (Pohlig-Hellman reduction and Shank's baby step/giant step algorithm) use

In [63]:
k = 12
R = k * Q
assert R.dlog(Q) == k

Setting the NIST curve `secp256k1`

In [64]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1
m = 115792089237316195423570985008687907852837564279074904382605163141518161494337  # order of the curve
ec = EC_Weierstrass(p, 0, 7, m)
ec.hex = True  # display points in hex form
ec.short = True # display points in short form
G = ec('02 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798')  # generator

Generate a (pseudorandom) key pair

In [65]:
secret_key = randint(2,m-1)
public_key = secret_key * G
secret_key, public_key

(23529166972251975810298992610444093284962271729068871731505428628592805361023,
 0x03449ef2ed30a8deb96e584a08c329adbf1be87ce40f1a0e7b4e86178682c41a9c)